# Simple Montecarlo Example

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
def all_vals(run, name):
    vs = []
    for st in run.getsteps():
        v = run.step(st).get_tensor(name).as_numpy()
        vs.append(v)
    return np.array(vs)

This notebook will show how to run Montecarlo to capture the inputs/outputs/intermediate layers of an inference run and compare it with the corresponding data from the training run

## Train a Simple Model

This very simple model learns the coefficients of a simple linear equation: `y_hat=Wx+b`. The data we feed to it follows the formula `y=5x+1`, so we should converge to `W==5` and `b==1`

In [17]:
!rm -rf ts_outputs/train
!rm -rf model
!python ./simplemodel_train.py --mean 0 --stddev 2 --batchsize 256 --epochs 1000

2019-04-26 16:47:18.797526: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Epoch: 0001 cost= 24848.214843750 W= -0.17271397 b= 0.66170704
Epoch: 0101 cost= 458.329559326 W= 4.337546 b= 0.80482745
Epoch: 0201 cost= 7.844162464 W= 4.918249 b= 0.88463634
Epoch: 0301 cost= 1.285287023 W= 4.989703 b= 0.9312211
Epoch: 0401 cost= 0.432488054 W= 4.9987516 b= 0.9588612
Epoch: 0501 cost= 0.155153424 W= 4.9996114 b= 0.97537035
Epoch: 0601 cost= 0.055579536 W= 4.999961 b= 0.9852583
Epoch: 0701 cost= 0.019923627 W= 5.000009 b= 0.9911774
Epoch: 0801 cost= 0.007133640 W= 5.0000134 b= 0.9947197
Epoch: 0901 cost= 0.002556968 W= 4.9999995 b= 0.99683964
Optimization Finished!
Training cost= 0.0009253842 W= 4.999998 b= 0.99809885


This will deposit a model in `./model`

In [14]:
!ls -l ./model/

total 48
-rw-r--r--  1 olg  ANT\Domain Users     67 Apr 26 14:49 checkpoint
-rw-r--r--  1 olg  ANT\Domain Users      8 Apr 26 14:49 model.data-00000-of-00001
-rw-r--r--  1 olg  ANT\Domain Users    142 Apr 26 14:49 model.index
-rw-r--r--  1 olg  ANT\Domain Users  35877 Apr 26 14:49 model.meta


And will deposit the Tornasole/Montecarlo data in `./ts_output/train/`


In [18]:
!ls -l ./ts_outputs/train/

total 0
drwxr-xr-x  104 olg  ANT\Domain Users  3328 Apr 26 16:47 e31c126f-2b8c-48a4-9e03-bfc36492770d


In [19]:
training_run_names = training_run_catalog.list_candidates()
for training_run_name in training_run_names:
    print( f'Training Run {training_run_name}')
    tr = LocalRun(training_run_name, os.path.join(tdir,training_run_name))

Training Run e31c126f-2b8c-48a4-9e03-bfc36492770d
None


In [30]:
tr.step(340).get_tensornames()[:10]

['Rank:0',
 'gradients/Pow_grad/Pow:0',
 'gradients/Pow_grad/mul:0',
 'gradients/sub_grad/Neg:0',
 'gradients/cost_grad/range/delta:0',
 'gradients/cost_grad/Tile:0',
 'gradients/sub_grad/tuple/control_dependency:0',
 'gradients/Y_hat_grad/Shape_1:0',
 'gradients/cost_grad/DynamicStitch:0',
 'gradients/grad_ys_0:0']

In [28]:
mul = tr.step(340).get_tensor('gradients/product_grad/Mul:0').as_numpy()

In [29]:
mul.shape

(256,)